# Exploratory Analysis
---

This code imports zipped data files from an S3 bucket, determines the shape and size of the data frame (Pandas), detects outliers/NAN/Null values, calculates collumn-wise mean/median/mode/stddev, and performs some correlation analysis on the variables.

# Dependencies & Globals
---

In [48]:
import pandas as pd
import boto3
import botocore
import sagemaker
import numpy as np
from scipy import stats
from violin_plot import violin_plot
import matplotlib.pyplot as plt
import hvplot.pandas
import seaborn as sns
import importlib

#Globals
filename_prefix = 'normal_02_'    # Filename prefix, for storing artifacts
store_artifacts = True            # True will save figures, tables, and dataframes to 
redownload_dataset = False         # True will redownload a copy of the dataset from S3. False uses the local Jupyter copy

#data_filename = "SWaT_Dataset_Attack_v1_01.parquet" # Input dataset filename name here
#data_filename = "SWaT_Dataset_Attack_v1_02.parquet"
#data_filename = "SWaT_Dataset_Normal_v1_01.parquet"
data_filename = "SWaT_Dataset_Normal_v1_02.parquet"

# Root bucket for this project and location of artifact storage
bucket = "exploratoryanalysis-swat"             # Bucket location for working directory
prefix = "analysis_artifacts"                   # Folder location for working directory
folder = "normal_02"

# S3 bucket where the original data is downloaded and stored
downloaded_data_bucket = "exploratoryanalysis-swat"    # Bucket location for data directory
downloaded_data_prefix = "dataset_unzipped"            # Folder location of data files

# Verify Bucket Connection
---

In [49]:
# Housekeeping, sagemaker gets execution role and reigon from this sagemaker instance
execution_role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Check bucket existence and permissions
def check_bucket_permission(bucket):
    # check if the bucket exists
    permission = False
    try:
        boto3.Session().client("s3").head_bucket(Bucket=bucket)
    except botocore.exceptions.ParamValidationError as e:
        print(
            "Hey! You either forgot to specify your S3 bucket"
            " or you gave your bucket an invalid name!"
        )
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "403":
            print(f"Hey! You don't have permission to access the bucket, {bucket}.")
        elif e.response["Error"]["Code"] == "404":
            print(f"Hey! Your bucket, {bucket}, doesn't exist!")
        else:
            raise
    else:
        permission = True
    return permission

if check_bucket_permission(bucket):
    print(f"Training input/output will be stored in: s3://{bucket}/{prefix}/{folder}")
if check_bucket_permission(downloaded_data_bucket):
    print(f"Downloaded training data will be read from s3://{downloaded_data_bucket}/{downloaded_data_prefix}")

Training input/output will be stored in: s3://exploratoryanalysis-swat/analysis_artifacts/normal_02
Downloaded training data will be read from s3://exploratoryanalysis-swat/dataset_unzipped


# Download Datafile or Open Existing Parquet Datafile
---

In [50]:
s3 = boto3.client("s3")

if redownload_dataset == True:  
    s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{data_filename}", data_filename)    # Download S3 data file to Sagemaker space
    
# Read file into Pandas Dataframe 
sensor_data = pd.read_parquet(data_filename)

# Check for NAN Values
---

In [51]:
# Locate NAN values and store indicies
nan_values = sensor_data[sensor_data.isna().any(axis=1)]
if nan_values.empty:
    pass
else:
    # Determine what we want to do with the missing fields
    print('There are NAN values in the data')

# Data Statistics
## Mean, Median, Std Dev, Unique Values, etc.
---

In [53]:
# Determine mean, median, std dev, etc. of each collumn
data_statistics = sensor_data.describe()
#print("Dataset statistics for each column:")
pd.set_option('display.max_columns', None)
#display(data_statistics)

if store_artifacts == True:
    filename = filename_prefix + 'data_statistics.csv'
    data_statistics.to_csv(filename, index=True)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

## Count of Unique Values per Column, Constant Variables

In [54]:
# What are the unique values and counts for each categorical column
unique_values = sensor_data.nunique(axis=0)
#display(unique_values)

if store_artifacts == True:
    filename = filename_prefix + 'unique_values_count.csv'
    unique_values.to_csv(filename, index=True)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

# Which columns have only a single unique value? (Constants)
constant_columns = unique_values.where(unique_values==1).dropna(how='all')
#display(constant_columns)

if store_artifacts == True:
    filename = filename_prefix + 'constant_values.csv'
    constant_columns.to_csv(filename, index=True)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

# Outliers
---

In [55]:
# Locate and classify outliers
# The collumns include two objects: a timestamp and a string. These cannot be analyzed for outliers, and will be removed by selecting only columns with numbers
numerical_sensor_data = sensor_data.select_dtypes(include='number')

# Z Score tells us how many standard deviations each value is from the mean. Anything outside of 3 STD deviations is an outlier and anything outside 2 is in the farthest 5% and could be cut
z_scores = numerical_sensor_data.apply(stats.zscore)

pd.set_option('display.max_columns', None)
#display(z_scores.head())

if store_artifacts == True:
    filename = filename_prefix + 'z_scores_matrix.parquet'
    z_scores.to_parquet(filename)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [56]:
# Interquartile Range (IQR) is another way to classify outliers. 
# IQR determines the range between the 25th and 75th percentile
# Then, anything that lies 1.5*(range) above or below these quartiles is considered an outlier
# If the data value is lower than the lower range or larger than the upper range, store the index of the value.
# Then count the number of outliers

IQR = data_statistics.loc['75%'] - data_statistics.loc['25%']
IQR = IQR.to_frame()
IQR = IQR.transpose()

# Define lower and upper ranges for each variable. Statistics does not run on timestamp or string objects. Only columns with numerical values.
lower_range = data_statistics.loc['25%'] - 1.5 * IQR
upper_range = data_statistics.loc['75%'] + 1.5 * IQR

# The format of a query needs to be "column label <query condition> @<variable name to be compared against>"
# 'FIT101<@aa' where aa is the variable that holds the limit value for this iteration
lower_queries = ['{}<@aa'.format(k) for k in lower_range.columns]
upper_queries = ['{}>@aa'.format(k) for k in upper_range.columns]

jj = 0
tot_upper_outl = 0
tot_lower_outl = 0
for column in IQR:
    # Lower boundary check
    aa = lower_range.iat[0,jj]                                        # ex: FIT101
    lower_outliers = numerical_sensor_data.query(lower_queries[jj])   # ex: 'FIT101<@aa'
    if not lower_outliers.empty:
        #print('Condition that triggered the IQR outlier: ', lower_queries[jj], aa)
        #print('Number of "less than" outliers', lower_outliers.shape[0], '\n')
        tot_lower_outl += lower_outliers.shape[0]
    
    # Upper Boundary Check
    aa = upper_range.iat[0,jj]                                        # ex: FIT101
    upper_outliers = numerical_sensor_data.query(upper_queries[jj])   # ex: 'FIT101>@aa'
    if not upper_outliers.empty:
        #print('Condition that triggered the IQR outlier: ', upper_queries[jj], aa)
        #print('Number of "greater than" outliers', upper_outliers.shape[0], '\n')
        tot_upper_outl += upper_outliers.shape[0]
    jj += 1
    
#print('Total number of outliers: ', tot_lower_outl+tot_upper_outl)

if store_artifacts == True:
    filename = filename_prefix + 'lower_IQR_outliers.parquet'
    lower_outliers.to_parquet(filename)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )
    
    filename = filename_prefix + 'upper_IQR_outliers.parquet'
    upper_outliers.to_parquet(filename)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

# Correlation
---

In [57]:
# Correlation analysis
# First, drop all the columns with constant values. These will not impact a correlation matrix.
variable_sensor_data = sensor_data.drop(constant_columns.index, axis=1)

correlation_matrix = variable_sensor_data.corr()
pd.set_option('display.max_columns', None)
#display(correlation_matrix.head())

# Upload correlation matrix CSV to S3 Artifacts
if store_artifacts == True:
    filename = filename_prefix + 'correlation_matrix.csv'
    correlation_matrix.to_csv(filename, index=True)

    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

In [58]:
# Plot the correlation matrix
fig, ax = plt.subplots()

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(correlation_matrix)));
ax.set_yticks(np.arange(len(correlation_matrix)));
ax.minorticks_off()
ax.grid(which='major', 
        color='dimgrey', 
        linestyle='-', 
        linewidth=1, 
        visible=True);
ax.set_xticklabels(correlation_matrix.columns);
ax.set_yticklabels(correlation_matrix.index);

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), 
         rotation=45, 
         ha="right",
         rotation_mode="anchor");

title = filename_prefix + "correlation_matrix"

ax.set_title(title)
fig.set_figwidth(10);
fig.set_figheight(10);
fig.tight_layout();
im = ax.imshow(correlation_matrix, cmap='seismic',vmin=-1,vmax=1);

cbar = ax.figure.colorbar(im, ax=ax, shrink=0.5);
cbar.ax.set_ylabel('Correlation Score', rotation=-90, va="bottom");
plt.tight_layout();
#plt.show()

fig

if store_artifacts == True:
    filename = title + '.png'
    plt.savefig(filename)
    s3.upload_file(
        Filename=filename,
        Bucket=bucket,
        Key=f"{prefix}/{folder}/{filename}"
    )

# Plots

## Boxplots

## Violin Plots

In [59]:
# Violin plot shows the distribution of the data. Here I show the attack vs normal on the same plot
# I had to add a row full of empty data to get the violin to do the split
variable_sensor_data["all"]=""    # We need to add an empty column for the split violin plot to work.
    

for ii in variable_sensor_data.columns:
    fig = plt.figure()
    
    if 'normal' in filename_prefix:
        split = False
        hue = None
    else:
        split = True
        hue = 'Attack'
    
    if ii != ("all") and ii != ("Timestamp") and ii != ("Attack"):
        title = filename_prefix + "violin_" + ii
        ax = violin_plot(df= variable_sensor_data, 
                         title= title, 
                         y= ii,
                         scale = 'count',
                         fig_width = 5,
                         fig_height = 5,
                         inner = None,
                         split = split,
                         hue = hue);
        #plt.show(ax)
        
        if store_artifacts == True:
            filename = title + '.png'
            #fig
            plt.savefig(filename)
            s3.upload_file(
                Filename=filename,
                Bucket=bucket,
                Key=f"{prefix}/{folder}/{filename}"
            )
        plt.close('all')



## Time Series Plotting

Using MatPlotLib

Using Seaborn

Using HVPLOT (and Bokeh)